In [1]:
$target = 'TCSValidate'
New-Item -Path Purge -Type Directory


    Directory: C:\Users\Nick\Projects\TheCommunitySolution\DataMigration

Mode                 LastWriteTime         Length Name
----                 -------------         ------ ----
d----           3/24/2025  9:47 PM                Purge



### Record Purge

Purges records from the target instance, in opposite order of the dependency tree.
Indiscremenately purges all records of the specified type, regardless of the state of the record.

#### SObjects
- Account
- Location
- Address
- Learning
- LearningProgram
- Academic Term
- ProgramTermApplnTimeline
- Scholarship__c
- IndividualApplication
- Opportunity
- Scholarship_Award__c
- Task
- Event

In [ ]:
@(
    #"Event", 
    #"Task", 
    #"Scholarship_Award__c", 
    "Opportunity", 
    "IndividualApplication"
    "Scholarship__c", 
    "ProgramTermApplnTimeline",
    "AcademicTerm",
    "LearningProgram",
    "Learning",
    "Address",
    "Location",
    "Account"
) | ForEach-Object {
    $result = sf data query -q "SELECT Id FROM $($_)" -o $target --json |
    ConvertFrom-Json | 
    Select-Object -ExpandProperty Result

    if ($result.totalSize -eq 0) {
        return
    }

    $result.records | Select-Object -Property Id | Export-Csv -Path "Purge\$($_).csv" -NoTypeInformation -Encoding UTF8
}

Get-ChildItem -Path Purge | Sort-Object -Property CreationTime | ForEach-Object {
    $sobject = $_.Name -replace '.csv', ''
    sf data delete bulk --file $_.FullName --sobject $sobject --target-org $target --json --wait 15 | Out-File -FilePath Purge\$sobject.json -Encoding UTF8
}

In [3]:
# Clean up
Remove-Item -Path Purge -Recurse -Force